# MIMIC-IV Discharge Notes Explorer

This notebook helps you explore clinical notes and extract summaries from the MIMIC-IV dataset.

**Key sections in discharge notes:**
- **Chief Complaint** - Why the patient came in
- **History of Present Illness** - Detailed story of the illness
- **Brief Hospital Course** - **THE SUMMARY** of what happened during hospitalization
- **Discharge Diagnosis** - Final diagnoses


In [1]:
import sqlite3
import pandas as pd
import re

# Connect to the database
DB_PATH = "discharge_notes.db"
conn = sqlite3.connect(DB_PATH)

# Quick overview
df_info = pd.read_sql_query("SELECT COUNT(*) as total_notes FROM discharge_notes", conn)
print(f"✅ Connected to database")
print(f"📊 Total discharge notes: {df_info['total_notes'][0]:,}")


✅ Connected to database
📊 Total discharge notes: 331,793


In [2]:
# Section extraction function
def extract_section(text, section_name):
    """Extract a specific section from a discharge note"""
    patterns = [
        rf'{section_name}[:\s]*\n(.*?)(?=\n[A-Z][A-Za-z\s]+:\s*\n|\n___|\Z)',
        rf'{section_name}[:\s]*\n(.*?)(?=\n[A-Z][A-Za-z\s]+[:\n]|\Z)',
    ]
    for pattern in patterns:
        match = re.search(pattern, text, re.DOTALL | re.IGNORECASE)
        if match:
            result = match.group(1).strip()
            if len(result) > 10:
                return result
    return None

print("✅ Section extraction functions loaded")


✅ Section extraction functions loaded


## View Sample Notes with Summaries


In [3]:
# View sample notes metadata
query = """
SELECT note_id, subject_id, hadm_id, note_type, charttime, LENGTH(text) as text_length
FROM discharge_notes
LIMIT 10
"""
pd.read_sql_query(query, conn)


,note_id,subject_id,hadm_id,note_type,charttime,text_length
0,10000032-DS-21,10000032,22595853,DS,2180-05-07 00:00:00,7530
1,10000032-DS-22,10000032,22841357,DS,2180-06-27 00:00:00,10922
2,10000032-DS-23,10000032,29079034,DS,2180-07-25 00:00:00,10608
3,10000032-DS-24,10000032,25742920,DS,2180-08-07 00:00:00,10348
4,10000084-DS-17,10000084,23052089,DS,2160-11-25 00:00:00,9293
5,10000117-DS-21,10000117,22927623,DS,2181-11-15 00:00:00,4637
6,10000117-DS-22,10000117,27988844,DS,2183-09-21 00:00:00,6288
7,10000248-DS-10,10000248,20600184,DS,2192-11-30 00:00:00,8031
8,10000560-DS-15,10000560,28979390,DS,2189-10-17 00:00:00,3525
9,10000764-DS-11,10000764,27897940,DS,2132-10-19 00:00:00,9949


In [4]:
# Get a single note and view its sections
query = "SELECT note_id, subject_id, hadm_id, text FROM discharge_notes LIMIT 1"
sample = pd.read_sql_query(query, conn).iloc[0]

print(f"📋 Note ID: {sample['note_id']}")
print(f"👤 Subject ID: {sample['subject_id']}")
print(f"🏥 Admission ID: {sample['hadm_id']}")
print(f"📏 Text length: {len(sample['text']):,} characters")

# Extract key sections
chief_complaint = extract_section(sample['text'], 'Chief Complaint')
summary = extract_section(sample['text'], 'Brief Hospital Course')
diagnosis = extract_section(sample['text'], 'Discharge Diagnosis')

print(f"\n{'='*60}")
print("📌 CHIEF COMPLAINT:")
print(chief_complaint or "[Not found]")

print(f"\n{'='*60}")
print("📊 DISCHARGE DIAGNOSIS:")
print(diagnosis[:500] if diagnosis else "[Not found]")

print(f"\n{'='*60}")
print("📝 SUMMARY (Brief Hospital Course):")
print(summary[:1500] if summary else "[Not found]")


📋 Note ID: 10000032-DS-21
👤 Subject ID: 10000032
🏥 Admission ID: 22595853
📏 Text length: 7,530 characters

📌 CHIEF COMPLAINT:
Worsening ABD distension and pain

📊 DISCHARGE DIAGNOSIS:
Ascites from Portal HTN

📝 SUMMARY (Brief Hospital Course):
___ HCV cirrhosis c/b ascites, hiv on ART, h/o IVDU, COPD, 
bioplar, PTSD, presented from OSH ED with worsening abd 
distension over past week and confusion.  

# Ascites - p/w worsening abd distension and discomfort for last 
week. likely ___ portal HTN given underlying liver disease, 
though no ascitic fluid available on night of admission. No 
signs of heart failure noted on exam. This was ___ to med 
non-compliance and lack of diet restriction. SBP negative


## Create Dataset: Notes + Summaries

Extract clinical notes paired with their **Brief Hospital Course** summaries.


In [ ]:
def get_notes_with_summaries(limit=100):
    """Get notes with their brief hospital course summaries"""
    query = f"SELECT note_id, subject_id, hadm_id, text FROM discharge_notes LIMIT {limit}"
    df = pd.read_sql_query(query, conn)
    
    results = []
    for _, row in df.iterrows():
        summary = extract_section(row['text'], 'Brief Hospital Course')
        chief_complaint = extract_section(row['text'], 'Chief Complaint')
        hpi = extract_section(row['text'], 'History of Present Illness')
        
        if summary:  # Only include if we found a summary
            results.append({
                'note_id': row['note_id'],
                'subject_id': row['subject_id'],
                'hadm_id': row['hadm_id'],
                'chief_complaint': chief_complaint,
                'history_of_present_illness': hpi,
                'full_text': row['text'],
                'summary': summary,
                'full_text_length': len(row['text']),
                'summary_length': len(summary)
            })
    
    return pd.DataFrame(results)

# Extract 100 notes with summaries
notes_df = get_notes_with_summaries(100)
print(f"✅ Extracted {len(notes_df)} notes with summaries")
notes_df[['note_id', 'chief_complaint', 'full_text_length', 'summary_length']].head(10)


In [ ]:
# View a specific note's full summary
idx = 0  # Change this to view different notes
note = notes_df.iloc[idx]

print(f"Note: {note['note_id']}")
print(f"Chief Complaint: {note['chief_complaint']}")
print(f"\n{'='*60}")
print("FULL SUMMARY (Brief Hospital Course):")
print(f"{'='*60}\n")
print(note['summary'])


## Search Notes by Condition/Keyword


In [ ]:
def search_notes(keyword, limit=10):
    """Search notes by keyword"""
    query = f"""
    SELECT note_id, subject_id, hadm_id, text 
    FROM discharge_notes 
    WHERE text LIKE '%{keyword}%'
    LIMIT {limit}
    """
    df = pd.read_sql_query(query, conn)
    
    results = []
    for _, row in df.iterrows():
        summary = extract_section(row['text'], 'Brief Hospital Course')
        diagnosis = extract_section(row['text'], 'Discharge Diagnosis')
        results.append({
            'note_id': row['note_id'],
            'subject_id': row['subject_id'],
            'discharge_diagnosis': diagnosis[:200] if diagnosis else None,
            'summary_preview': summary[:300] + '...' if summary and len(summary) > 300 else summary
        })
    
    return pd.DataFrame(results)

# Search for pneumonia cases
pneumonia_notes = search_notes('pneumonia', limit=5)
pneumonia_notes


## Export Notes with Summaries to CSV


In [ ]:
# Export a larger dataset (change limit as needed)
export_df = get_notes_with_summaries(1000)

# Save to CSV
export_df.to_csv('notes_with_summaries.csv', index=False)
print(f"✅ Exported {len(export_df)} notes to notes_with_summaries.csv")

# Statistics
print(f"\n📊 Dataset Statistics:")
print(f"   Average full text length: {export_df['full_text_length'].mean():,.0f} chars")
print(f"   Average summary length: {export_df['summary_length'].mean():,.0f} chars")
print(f"   Compression ratio: {export_df['full_text_length'].mean() / export_df['summary_length'].mean():.1f}x")


## Run Custom SQL Queries


In [ ]:
# Run any SQL query
query = """
SELECT 
    COUNT(*) as total_notes,
    AVG(LENGTH(text)) as avg_length,
    MIN(LENGTH(text)) as min_length,
    MAX(LENGTH(text)) as max_length
FROM discharge_notes
"""
pd.read_sql_query(query, conn)
